## Manga PDF'leme

In [1]:
import os
import requests
from reportlab.lib.pagesizes import letter
from reportlab.pdfgen import canvas

In [ ]:
def get_image_urls(manga_chapter_link):
    data = requests.get(manga_chapter_link)
    long_string = data.text
    start_marker = "<p id=arraydata style=display:none>"
    end_marker = "</p>\r"

    # Find the position of the first occurrence of start_marker
    start_index = long_string.find(start_marker)

    # Check if start_marker is found
    if start_index != -1:
        # Find the position of the first occurrence of end_marker after start_index
        end_index = long_string.find(end_marker, start_index + len(start_marker))
        
        # Check if end_marker is found after start_marker
        if end_index != -1:
            # Extract the substring between start_index and end_index
            result = long_string[start_index + len(start_marker):end_index]
            result = result.split(",")
            print("found " + str(len(result)) + " png files")
        else:
            print("end_marker not found after " + start_marker)
    else:
        print(start_marker + " not found")

    return result

In [ ]:
def download_images(image_urls, manga_name, chapter):
    # Create a directory to save the downloaded images
    if not os.path.exists('downloaded_images'):
        os.makedirs('downloaded_images')
    if not os.path.exists('downloaded_images/'+manga_name):
        os.makedirs('downloaded_images/'+manga_name)
    if not os.path.exists('downloaded_images/'+manga_name+"/"+chapter):
        os.makedirs('downloaded_images/'+manga_name+"/"+chapter)

    # Download and save each image locally
    for i, image_url in enumerate(image_urls):
        response = requests.get(image_url)
        if response.status_code == 200:
            with open('downloaded_images/'+manga_name+"/"+chapter+'/image_'+str(i+ 1)+'.png', 'wb') as image_file:
                image_file.write(response.content)
        else:
            print("failed to download page", i + 1)

In [ ]:
def create_PDF_from_images(image_urls, manga_name, chapter):
    if not os.path.exists('Manga_PDFs'):
        os.makedirs('Manga_PDFs')
    if not os.path.exists('Manga_PDFs/'+manga_name):
        os.makedirs('Manga_PDFs/'+manga_name)
    
    # Output PDF file path
    output_pdf_path = 'Manga_PDFs/'+manga_name+"/"+chapter + '.pdf'

    # Create a PDF document
    c = canvas.Canvas(output_pdf_path, pagesize=letter)

    # Loop through the local image files and add each image as a page
    for i in range(len(image_urls)):
        image_path = 'downloaded_images/'+manga_name+"/"+chapter+'/image_'+str(i+ 1)+'.png'
        if os.path.exists(image_path):
            # Define the width and height of the page (adjust as needed)
            width, height = letter
            # Add the image to the PDF page, maintaining the aspect ratio
            c.drawImage(image_path, 0, 0, width, height, preserveAspectRatio=True)
            c.showPage()

    # Add a white blank page at the end
    c.showPage()

    # Save the PDF document
    c.save()

    print(f'PDF saved as "{output_pdf_path}"')

## 1

In [ ]:
def download_manga(manga_link, manga_name, chapter, end_chapter):
    while True:
        manga_chapter_link = manga_link + chapter

        print("processing chapter:", chapter)

        try:
            image_urls = get_image_urls(manga_chapter_link)
        except:
            print("error during geting image urls")
            break

        try:
            download_images(image_urls, manga_name, chapter)
        except:
            print("error during dowloading images")
            break

        try:
            create_PDF_from_images(image_urls, manga_name, chapter)
        except:
            print("error during creating pdf")
            break

        chapter = str(int(chapter) + 1)

        if chapter == end_chapter:
            break

In [ ]:
manga_link = "https://mangapanda.in/how-to-live-as-an-unlicensed-healer-chapter-"
manga_name = "deneme"
chapter = "1"
end_chapter = "2"

download_manga(manga_link, manga_name, chapter, end_chapter)

## 2

In [ ]:
def download_manga_with_list(manga_link, manga_name, chapter_list):
    for chapter in chapter_list:
        manga_chapter_link = manga_link + chapter

        print("processing chapter:", chapter)

        try:
            image_urls = get_image_urls(manga_chapter_link)
        except:
            print("error during geting image urls")
            break

        try:
            download_images(image_urls, manga_name, chapter)
        except:
            print("error during dowloading images")
            break

        try:
            create_PDF_from_images(image_urls, manga_name, chapter)
        except:
            print("error during creating pdf")
            break

In [ ]:
manga_link = "https://mangapanda.in/the-reincarnated-sage-who-possesses-the-mark-of-the-emperor-that-commands-obedience-from-monsters-quietly-becoming-the-strongest-adventurer-with-the-magic-from-his-previous-life-alongside-his-monster-chapter-"
manga_name = "deneme2"
chapter_list = ["1.1", "1.2", "2.1", "2.2", "3.1", "3.2", "3.3", "4.1", "4.2", "5", "6", "6.1", "7", "8", "9", "10", "11", "12", "13", "14", "15"]

download_manga_with_list(manga_link, manga_name, chapter_list)